# **Model Tuning**

In [ ]:
# Required packages

!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U seaborn
!pip install -U evaluate
!pip install -U emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 32.5 MB/s eta 0:00:00


# Data Preprocessing

In [ ]:
import pandas as pd
import re

def load_dataset(file_path, label):
    """
    Load the dataset from a file and add a numeric label to each entry.

    Args:
    - file_path (str): Path to the dataset file containing text data. Each line in the file
      should represent one entry (e.g., a tweet or text sample).
    - label (int): Numeric label to associate with all entries in the dataset. Typically,
      1 for positive sentiment and 0 for negative sentiment.

    Returns:
    - pd.DataFrame: A Pandas DataFrame with the following columns:
        - 'text': The textual data loaded from the file.
        - 'label': The numeric label provided, applied to all entries in the dataset.
    """
    # Open the file at the given path with UTF-8 encoding and read all lines into a list.
    with open(file_path, 'r', encoding='utf-8') as file:
        tweets = file.readlines()

    # Create a DataFrame from the list of tweets with a single column named 'text'.
    df = pd.DataFrame(tweets, columns=['text'])

    # Add a new column 'label' to the DataFrame and assign the provided label value to all rows.
    df['label'] = label

    # Return the resulting DataFrame.
    return df

def clean_tweet(tweet):
    """
    Clean a single tweet by removing unnecessary parts such as placeholders, special characters,
    and normalizing text.

    Args:
    - tweet (str): The raw tweet text.

    Returns:
    - str: The cleaned tweet text.
    """
    tweet = re.sub(r"<user>", "", tweet)  # Remove <user>
    tweet = re.sub(r"<url>", "", tweet)  # Remove <url>
    tweet = re.sub(r"\.{2,}", "", tweet)  # Remove ellipses (two or more dots)
    tweet = re.sub(r"\s+", " ", tweet)  # Normalize whitespace
    tweet = re.sub(r"\b\d+(?:st|nd|rd|th)?\b", "", tweet)  # Remove numbers and ordinals
    return tweet.strip()

# Paths to your datasets
# Specify file paths for negative and positive sentiment training data.
train_neg_path = 'drive/MyDrive/Colab Notebooks/train_neg_full.txt'  # Path to negative tweets dataset
train_pos_path = 'drive/MyDrive/Colab Notebooks/train_pos_full.txt'  # Path to positive tweets dataset

# Load datasets with numeric labels
# Assign label 0 to negative tweets and label 1 to positive tweets using the `load_dataset` function.
df_neg = load_dataset(train_neg_path, 0)  # Negative tweets
df_pos = load_dataset(train_pos_path, 1)  # Positive tweets

# Combine datasets
# Concatenate the positive and negative datasets into a single DataFrame for easier processing.
df = pd.concat([df_neg, df_pos], ignore_index=True)

# Clean tweets
# Apply the `clean_tweet` function to each tweet in the 'text' column of the DataFrame.
df['text'] = df['text'].apply(clean_tweet)

# Remove duplicates
# Drop duplicate entries based on the 'text' column to avoid redundant data.
df = df.drop_duplicates(subset='text').copy()

# Add a label_name column based on the label
# Create a new column 'label_name' to map numeric labels to human-readable categories.
# Label 1 (positive sentiment) maps to "positive", and label 0 (negative sentiment) maps to "negative".
df['label_name'] = df['label'].apply(lambda x: 'positive' if x == 1 else 'negative')

In [ ]:
df['label_name'].value_counts()

,count
label_name,
sad,90744
happy,90013


# Train-test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Set a fixed random_state for reproducibility
RANDOM_STATE = 42

# Split the dataset into training and testing sets
# 30% of the data is reserved for testing, while the remaining 70% is used for training.
train, test = train_test_split(df, test_size=0.3, random_state=RANDOM_STATE)

# Further split the test set into validation and test sets
# One-third of the test set is used for validation, while the remaining two-thirds are kept as the final test set.
test, validation = train_test_split(test, test_size=1/3, random_state=RANDOM_STATE)

# Print the shapes of the resulting datasets (train, test, and validation)
train.shape, test.shape, validation.shape

((126529, 4), (36152, 4), (18076, 4))

In [ ]:
from datasets import Dataset, DatasetDict

# Create a DatasetDict object to organize the data into train, test, and validation splits
dataset = DatasetDict({
    'train': Dataset.from_pandas(train, preserve_index=False),
    'test': Dataset.from_pandas(test, preserve_index=False),
    'validation': Dataset.from_pandas(validation, preserve_index=False)
})

# Display the DatasetDict object
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'Words per Tweet'],
        num_rows: 126529
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'Words per Tweet'],
        num_rows: 36152
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'Words per Tweet'],
        num_rows: 18076
    })
})

# Tokenization of the Data

In [ ]:
from transformers import AutoTokenizer

# Specify the pre-trained model checkpoint to use for tokenization
# Specify if BERT or BETweet
model_ckpt = "vinai/bertweet-base"
# model_ckpt = "bert-base-uncased"

# Load the tokenizer associated with the specified model checkpoint
# The tokenizer will handle tokenizing raw text into input IDs and tokens suitable for the model.
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def tokenize(batch):
    """
    Tokenize a batch of text data using the pre-loaded tokenizer.

    Args:
    - batch (dict): A batch of data containing a 'text' key with raw text to tokenize.

    Returns:
    - dict: Tokenized output including input IDs, attention masks, and other relevant fields.
    """
    # Use the tokenizer to process the text data in the batch
    # - padding=True ensures all sequences in the batch are padded to the same length.
    # - truncation=True ensures that sequences longer than the model's max length are truncated.
    temp = tokenizer(batch['text'], padding=True, truncation=True)

    return temp

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[0, 1508, 839, 53, 5379, 5417, 34, 13101, 9, 16271, 57, 1982, 202, 156, 2], [0, 16, 6, 161, 1462, 136, 77, 4, 8210, 2193, 44499, 2, 1, 1, 1]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}


In [ ]:
# Apply the `tokenize` function to the entire dataset
emotion_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/126529 [00:00<?, ? examples/s]

Map:   0%|          | 0/36152 [00:00<?, ? examples/s]

Map:   0%|          | 0/18076 [00:00<?, ? examples/s]

In [ ]:
# Create a mapping from label names (e.g., 'positive', 'negative') to their numeric IDs
# Iterate through the training dataset to extract label names and their corresponding numeric labels.
label2id = {x['label_name']: x['label'] for x in dataset['train']}

# Create the reverse mapping: from numeric IDs to label names
# Swap the keys and values from the `label2id` dictionary to create `id2label`.
id2label = {v: k for k, v in label2id.items()}

# Hyper-parameter Tuning

In [ ]:
from transformers import AutoModel
import torch

In [ ]:
# Use the Hugging Face `AutoModel` class to load the model associated with the specified checkpoint.
# This initializes the model, ready for further fine-tuning or inference tasks.
model = AutoModel.from_pretrained(model_ckpt)
model

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score
from itertools import product
import torch

# Model configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameter grid
# A grid of hyperparameters to search through, including:
# - learning rates
# - batch sizes
# - number of epochs
# - weight decay for regularization
learning_rates = [2e-5, 3e-5, 5e-5]
batch_sizes = [32, 64]
num_epochs = [2, 3]
weight_decays = [0.01, 0.1]

# Generate all possible combinations of the hyperparameters using Cartesian product
hyperparameter_combinations = list(product(learning_rates, batch_sizes, num_epochs, weight_decays))

# Metric computation function
def compute_metrics(pred):
    """
    Compute evaluation metrics for the model: accuracy and F1-score.

    Args:
    - pred: Predictions output from the model during evaluation.

    Returns:
    - dict: A dictionary containing accuracy and weighted F1-score.
    """
    labels = pred.label_ids  # True labels
    preds = pred.predictions.argmax(-1)  # Predicted class labels from logits
    f1 = f1_score(labels, preds, average="weighted")  # Weighted F1-score
    acc = accuracy_score(labels, preds)  # Accuracy
    return {"accuracy": acc, "f1": f1}

# Initialize a list to store evaluation results for each hyperparameter combination
results = []

# Iterate over all combinations of hyperparameters
for lr, batch_size, epochs, weight_decay in hyperparameter_combinations:
    print(f"Training with: lr={lr}, batch_size={batch_size}, epochs={epochs}, weight_decay={weight_decay}")

    # Define training arguments for the current hyperparameter combination
    training_args = TrainingArguments(
        output_dir=f"./results_lr_{lr}_bs_{batch_size}_ep_{epochs}_wd_{weight_decay}",  # Directory for logs and outputs
        overwrite_output_dir=True,  # Overwrite output directory if it exists
        num_train_epochs=epochs,  # Number of training epochs
        learning_rate=lr,  # Learning rate for optimization
        per_device_train_batch_size=batch_size,  # Batch size per device for training
        per_device_eval_batch_size=batch_size,  # Batch size per device for evaluation
        weight_decay=weight_decay,  # Weight decay for regularization
        evaluation_strategy="epoch",  # Evaluate at the end of each epoch
        save_strategy="no",  # Do not save checkpoints to reduce overhead
        disable_tqdm=False,  # Display progress bars during training
        logging_dir=f"./logs_lr_{lr}_bs_{batch_size}_ep_{epochs}_wd_{weight_decay}",  # Directory for logging
        load_best_model_at_end=False  # Do not load the best model automatically
    )

    # Initialize model configuration with label mappings
    config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)

    # Load the pre-trained model for sequence classification
    model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

    # Initialize the Trainer for training and evaluation
    trainer = Trainer(
        model=model,
        args=training_args,  # Training arguments
        compute_metrics=compute_metrics,  # Function to compute accuracy and F1-score
        train_dataset=emotion_encoded['train'],  # Training dataset
        eval_dataset=emotion_encoded['validation'],  # Validation dataset
        tokenizer=tokenizer  # Tokenizer used for preprocessing
    )

    # Train the model with the current hyperparameter combination
    trainer.train()

    # Evaluate the model on the validation dataset
    eval_results = trainer.evaluate()
    eval_results["lr"] = lr  # Add learning rate to the results
    eval_results["batch_size"] = batch_size  # Add batch size
    eval_results["epochs"] = epochs  # Add number of epochs
    eval_results["weight_decay"] = weight_decay  # Add weight decay value
    results.append(eval_results)  # Store the results for this combination

# Print all evaluation results for each hyperparameter combination
for result in results:
    print(result)

# Find the best combination based on the highest F1-score
best_result = max(results, key=lambda x: x["eval_f1"])
print("Best hyperparameters:", best_result)

Training with: lr=2e-05, batch_size=32, epochs=2, weight_decay=0.01


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-40088fcdb858>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` paramete

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.259700,0.263632,0.894335,0.894271
2,0.189800,0.269222,0.896105,0.896103


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.287000,0.277447,0.890020,0.890014
2,0.195000,0.285183,0.893173,0.893173
3,0.109900,0.353133,0.890739,0.890739


Training with: lr=5e-05, batch_size=64, epochs=3, weight_decay=0.1


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-40088fcdb858>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.285700,0.277775,0.890407,0.890406
2,0.191100,0.284213,0.893948,0.893945
3,0.111000,0.354364,0.891403,0.891403


{'eval_loss': 0.2692217230796814, 'eval_accuracy': 0.8961053330382828, 'eval_f1': 0.8961025563902978, 'eval_runtime': 15.0646, 'eval_samples_per_second': 1199.896, 'eval_steps_per_second': 37.505, 'epoch': 2.0, 'lr': 2e-05, 'batch_size': 32, 'epochs': 2, 'weight_decay': 0.01}
{'eval_loss': 0.2681236267089844, 'eval_accuracy': 0.8969351626466032, 'eval_f1': 0.8969336343359718, 'eval_runtime': 15.042, 'eval_samples_per_second': 1201.699, 'eval_steps_per_second': 37.561, 'epoch': 2.0, 'lr': 2e-05, 'batch_size': 32, 'epochs': 2, 'weight_decay': 0.1}
{'eval_loss': 0.3224394917488098, 'eval_accuracy': 0.8948329276388581, 'eval_f1': 0.8948329414790966, 'eval_runtime': 15.0679, 'eval_samples_per_second': 1199.638, 'eval_steps_per_second': 37.497, 'epoch': 3.0, 'lr': 2e-05, 'batch_size': 32, 'epochs': 3, 'weight_decay': 0.01}
{'eval_loss': 0.3227744996547699, 'eval_accuracy': 0.8933945563177694, 'eval_f1': 0.8933945416357014, 'eval_runtime': 15.0447, 'eval_samples_per_second': 1201.489, 'eval_s